###Fuente de la data

Los micro-datos gestionados en este notebook son tomados de www.datos.gov.co y corresponden a datos cuya fuente primaria es el Ministerio de Educación Nacional

In [1]:
import pandas as pd
import gc

###Archivo sty3-c395.csv

Contiene información sobre la matrícula estadística de Educación Preescolar Básica y Media de Colombia del corte 2018 al 2023.Agregado de matricula por establecimiento educativo, que contiene las variables del establecimiento (sede, zona, jornada, grado, metodología, carácter y sector CONPES) y de la matricula (edad, género y grupo étnico)

In [2]:
# Especificar las columnas que deseas cargar
columnas_necesarias = ['anno_inf','departamento', 'secretaria', 'cod_dane_municipio', 'municipio','codigo_dane',
       'nombre_establecimiento', 'sector', 'calendario', 'nombre_sede','zona','tipo_jornada', 'grado', 'metodologia', 'caracter',
        'especialidad', 'edad', 'genero', 'grupo_etnico', 'total_matricula' ]

# Lista para acumular los fragmentos de datos
data_chunks = []

# Leer el archivo en fragmentos (chunks) de 500,000 filas, solo cargando las columnas necesarias
chunksize = 500000
for chunk in pd.read_csv("https://www.datos.gov.co/resource/sty3-c395.csv?$limit=6200000",
                         dtype={'cod_dane_municipio': str},
                         chunksize=chunksize,
                         usecols=columnas_necesarias):
    # Procesar o modificar cada chunk si es necesario
    # Agregar el fragmento a la lista
    data_chunks.append(chunk)

# Concatenar todos los fragmentos en un solo DataFrame
df = pd.concat(data_chunks, ignore_index=True)

# Limpiar la memoria liberando la lista de fragmentos
del data_chunks
gc.collect()

# Mostrar las primeras filas del DataFrame Dask
df.head()

,anno_inf,departamento,secretaria,cod_dane_municipio,municipio,codigo_dane,nombre_establecimiento,sector,calendario,nombre_sede,zona,tipo_jornada,grado,metodologia,caracter,especialidad,edad,genero,grupo_etnico,total_matricula
0,2012,"CAPITAL BOGOTÁ, D.C.",BOGOTÁ,11001,"BOGOTÁ, D.C.",311001048011,LIC MALLERLAND,NO_OFICIAL,A,LIC MALLERLAND,Urbana,Completa,Cuarto,Educación tradicional,No Aplica,No aplica,10,Masculino,No aplica,8
1,2012,"CAPITAL BOGOTÁ, D.C.",BOGOTÁ,11001,"BOGOTÁ, D.C.",311001088250,GIMN NUEVA VILLA MAYOR,NO_OFICIAL,A,GIMN NUEVA VILLA MAYOR,Urbana,Completa,Cuarto,Educación tradicional,No Aplica,No aplica,10,Masculino,No aplica,7
2,2012,"CAPITAL BOGOTÁ, D.C.",BOGOTÁ,11001,"BOGOTÁ, D.C.",311001048011,LIC MALLERLAND,NO_OFICIAL,A,LIC MALLERLAND,Urbana,Completa,Cuarto,Educación tradicional,No Aplica,No aplica,11,Femenino,No aplica,1
3,2012,ANTIOQUIA,TURBO,5837,TURBO,205837001880,I.E. SAN VICENTE DEL CONGO,OFICIAL,A,E.R. MANTAGORDA,Rural,Completa,Cuarto,Educación tradicional,No Aplica,No aplica,11,Femenino,No aplica,1
4,2012,"CAPITAL BOGOTÁ, D.C.",BOGOTÁ,11001,"BOGOTÁ, D.C.",311001048011,LIC MALLERLAND,NO_OFICIAL,A,LIC MALLERLAND,Urbana,Completa,Cuarto,Educación tradicional,No Aplica,No aplica,11,Masculino,No aplica,2


##Revisión y limpieza para integrarlo a la base de datos

### - Resumen de la estructura del dataset

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6200000 entries, 0 to 6199999
Data columns (total 20 columns):
 #   Column                  Dtype 
---  ------                  ----- 
 0   anno_inf                int64 
 1   departamento            object
 2   secretaria              object
 3   cod_dane_municipio      object
 4   municipio               object
 5   codigo_dane             int64 
 6   nombre_establecimiento  object
 7   sector                  object
 8   calendario              object
 9   nombre_sede             object
 10  zona                    object
 11  tipo_jornada            object
 12  grado                   object
 13  metodologia             object
 14  caracter                object
 15  especialidad            object
 16  edad                    int64 
 17  genero                  object
 18  grupo_etnico            object
 19  total_matricula         int64 
dtypes: int64(4), object(16)
memory usage: 946.0+ MB


### Eliminación de columnas irrelevantes para el proyecto

In [4]:
description_col = {
    "anno_inf": "Año del reporte de Información",
    "cod_dane_departamento": "Código del Departamento de la Sede",
    "departamento": "Departamento de la Sede",
    "cod_secretaria": "Código de la Secretaría de Educación donde ha sido atendido el estudiante",
    "secretaria": "Secretaría de Educación donde ha sido atendido el estudiante",
    "cod_dane_municipio": "Código del Municipio de la Sede",
    "municipio": "Municipio de la Sede",
    "codigo_dane": "Municipio Dane Establecimiento",
    "nombre_establecimiento": "Nombre del Establecimiento",
    "cod_sector": "Código del Sector de atención del Establecimiento",
    "sector": "Sector de atención del Establecimiento",
    "cod_calendario": "Código del Calendario del Establecimiento",
    "calendario": "Calendario del Establecimiento",
    "codigo_dane_sede": "Código Dane de la Sede",
    "nombre_sede": "Nombre de la Sede",
    "cod_zona": "Código de la Zona donde se encuentra la sede",
    "zona": "Zona donde se encuentra la sede",
    "cod_tipo_jornada": "Código del tipo de Jornada de la Sede",
    "tipo_jornada": "Tipo de Jornada de la Sede",
    "codigo_grado": "Código grado",
    "grado": "Descriptivo del grado",
    "cod_metodologia": "Código de la metodología",
    "metodologia": "Metodología",
    "cod_caracter": "Código del Carácter del Establecimiento",
    "caracter": "Carácter del Establecimiento",
    "cod_especialidad": "Código de la Especialidad del Establecimiento",
    "especialidad": "Especialidad del Establecimiento",
    "edad": "Edad calculada con referencia al inicio del calendario escolar",
    "cod_genero": "Letra que identifica el Sexo Biológico",
    "genero": "Sexo Biológico",
    "cod_grupo_etnico": "Código del Grupo Étnico",
    "grupo_etnico": "Grupo Étnico",
    "cod_sector_conpes": "Código del sector para asignación de recursos CONPES",
    "sector_conpes": "Sector para asignación de recursos CONPES",
    "total_matricula": "Total de Matrícula 2018"
}

In [5]:
# Dejar columnas importantes
relevant_cols = ['anno_inf','departamento', 'cod_dane_municipio', 'municipio','codigo_dane',
       'nombre_establecimiento', 'sector', 'calendario', 'nombre_sede','zona','tipo_jornada', 'grado',
        'especialidad', 'edad', 'genero', 'grupo_etnico', 'total_matricula' ]
# Crear nuevo dataframe dask con las columnas importantes
preschool_primary_secondary_education_Mineducacion = df[relevant_cols]

# Eliminar df_dask, porque no se va a volver a utilizar
del df
gc.collect()


0

In [6]:
preschool_primary_secondary_education_Mineducacion.isnull().sum()

,0
anno_inf,0
departamento,6828
cod_dane_municipio,0
municipio,6828
codigo_dane,0
nombre_establecimiento,0
sector,0
calendario,0
nombre_sede,0
zona,0


### Gestionar valores nulos

In [7]:
# Identificar las filas que tienen al menos un valor nulo
nulos = preschool_primary_secondary_education_Mineducacion[
    preschool_primary_secondary_education_Mineducacion.isnull().any(axis=1)
]

# Mostrar las primeras filas (debe computarse)
nulos.head()

,anno_inf,departamento,cod_dane_municipio,municipio,codigo_dane,nombre_establecimiento,sector,calendario,nombre_sede,zona,tipo_jornada,grado,especialidad,edad,genero,grupo_etnico,total_matricula
7424,2012,NaN,5380,NaN,105380000063,I. E. BERNARDO ARANGO MACIAS,OFICIAL,A,I. E. BERNARDO ARANGO MACIAS,Urbana,Tarde,Noveno,No aplica,13,Masculino,No aplica,4
8398,2012,NaN,5380,NaN,105380000063,I. E. BERNARDO ARANGO MACIAS,OFICIAL,A,I. E. BERNARDO ARANGO MACIAS,Urbana,Tarde,Noveno,No aplica,15,Masculino,No aplica,16
8401,2012,NaN,5380,NaN,105380000063,I. E. BERNARDO ARANGO MACIAS,OFICIAL,A,I. E. BERNARDO ARANGO MACIAS,Urbana,Tarde,Noveno,No aplica,15,Femenino,No aplica,15
8402,2012,NaN,5380,NaN,105380000063,I. E. BERNARDO ARANGO MACIAS,OFICIAL,A,I. E. BERNARDO ARANGO MACIAS,Urbana,Tarde,Noveno,No aplica,14,Masculino,No aplica,34
8403,2012,NaN,5380,NaN,105380000063,I. E. BERNARDO ARANGO MACIAS,OFICIAL,A,I. E. BERNARDO ARANGO MACIAS,Urbana,Tarde,Noveno,No aplica,14,Femenino,No aplica,36


In [8]:
len(nulos)

6828

Es posible utilizar la columna codigo_dane, que se supone es única para cada establecimiento educativo, para reemplazar los valores nulos de las columnas departamento y municipio, basándote en las filas donde esos valores ya están presentes. Este enfoque se llama imputación por correspondencia o imputación basada en relaciones existentes.

In [9]:
# Crear un mapeo basado en codigo_dane:
codigo_dane_map = preschool_primary_secondary_education_Mineducacion.dropna(subset=['departamento', 'municipio']).groupby('codigo_dane')[['departamento', 'municipio']].first()

# Identificar las filas con valores nulos en esas tres columnas:
nulos_df = preschool_primary_secondary_education_Mineducacion[
    preschool_primary_secondary_education_Mineducacion['departamento'].isnull() |
    preschool_primary_secondary_education_Mineducacion['municipio'].isnull()
].copy()

# Rellenar los valores nulos utilizando el mapeo:
nulos_df.loc[:, 'departamento'] = nulos_df['codigo_dane'].map(codigo_dane_map['departamento']).fillna(nulos_df['departamento'])
nulos_df.loc[:, 'municipio'] = nulos_df['codigo_dane'].map(codigo_dane_map['municipio']).fillna(nulos_df['municipio'])
#Actualizar las filas del DataFrame original con los valores corregidos:
preschool_primary_secondary_education_Mineducacion.update(nulos_df)


In [10]:
preschool_primary_secondary_education_Mineducacion.head()

,anno_inf,departamento,cod_dane_municipio,municipio,codigo_dane,nombre_establecimiento,sector,calendario,nombre_sede,zona,tipo_jornada,grado,especialidad,edad,genero,grupo_etnico,total_matricula
0,2012,"CAPITAL BOGOTÁ, D.C.",11001,"BOGOTÁ, D.C.",311001048011,LIC MALLERLAND,NO_OFICIAL,A,LIC MALLERLAND,Urbana,Completa,Cuarto,No aplica,10,Masculino,No aplica,8
1,2012,"CAPITAL BOGOTÁ, D.C.",11001,"BOGOTÁ, D.C.",311001088250,GIMN NUEVA VILLA MAYOR,NO_OFICIAL,A,GIMN NUEVA VILLA MAYOR,Urbana,Completa,Cuarto,No aplica,10,Masculino,No aplica,7
2,2012,"CAPITAL BOGOTÁ, D.C.",11001,"BOGOTÁ, D.C.",311001048011,LIC MALLERLAND,NO_OFICIAL,A,LIC MALLERLAND,Urbana,Completa,Cuarto,No aplica,11,Femenino,No aplica,1
3,2012,ANTIOQUIA,5837,TURBO,205837001880,I.E. SAN VICENTE DEL CONGO,OFICIAL,A,E.R. MANTAGORDA,Rural,Completa,Cuarto,No aplica,11,Femenino,No aplica,1
4,2012,"CAPITAL BOGOTÁ, D.C.",11001,"BOGOTÁ, D.C.",311001048011,LIC MALLERLAND,NO_OFICIAL,A,LIC MALLERLAND,Urbana,Completa,Cuarto,No aplica,11,Masculino,No aplica,2


In [11]:
preschool_primary_secondary_education_Mineducacion.shape

(6200000, 17)

In [12]:
preschool_primary_secondary_education_Mineducacion.isnull().sum()

,0
anno_inf,0
departamento,853
cod_dane_municipio,0
municipio,853
codigo_dane,0
nombre_establecimiento,0
sector,0
calendario,0
nombre_sede,0
zona,0


In [13]:
# Llamar al recolector de basura
gc.collect()

0

In [14]:
# Los 865 nulos que quedaron despues de la imputación por correspondencia o imputación basada en relaciones existentes se eliminan
preschool_primary_secondary_education_Mineducacion = preschool_primary_secondary_education_Mineducacion.dropna(subset=['departamento', 'municipio'])

In [15]:
preschool_primary_secondary_education_Mineducacion[['departamento', 'municipio']].isnull().sum()

,0
departamento,0
municipio,0


- Verificar anos que cubre la informacion

In [16]:
preschool_primary_secondary_education_Mineducacion['anno_inf'].unique()

array([2012, 2023, 2022])

### Estandarización de categorizaciones

La estandarización de categorizaciones es el proceso de uniformizar y normalizar los valores de las categorías en un conjunto de datos para asegurar la consistencia y evitar discrepancias. Esto es crucial para la calidad y precisión de los análisis

In [17]:
categorical_col = preschool_primary_secondary_education_Mineducacion.select_dtypes(include=['object']).columns.tolist()

In [18]:
# Imprimir categorías únicas para columnas de tipo object
for column in categorical_col:
    print(f"Categorías en la columna '{column}':")
    print(preschool_primary_secondary_education_Mineducacion[column].unique())
    print()

Categorías en la columna 'departamento':
['CAPITAL BOGOTÁ, D.C.' 'ANTIOQUIA' 'CALDAS' 'TOLIMA' 'NORTE DE SANTANDER'
 'SANTANDER' 'BOLÍVAR' 'ARAUCA' 'CESAR' 'CUNDINAMARCA' 'ATLÁNTICO'
 'CÓRDOBA' 'META' 'HUILA' 'BOYACÁ' 'SUCRE' 'GUAJIRA' 'MAGDALENA' 'CAUCA'
 'NARIÑO' 'VALLE DEL CAUCA' 'PUTUMAYO' 'GUAVIARE' 'VAUPÉS' 'CAQUETÁ'
 'VICHADA' 'AMAZONAS' 'CHOCÓ' 'GUAINÍA'
 'ARCHIPIÉLAGO DE SAN ANDRES,PROVIDENCIA Y SANTA CATALINA' 'RISARALDA'
 'CASANARE' 'QUINDIO']

Categorías en la columna 'cod_dane_municipio':
['11001' '5837' '5686' ... '97889' '94885' '91430']

Categorías en la columna 'municipio':
['BOGOTÁ, D.C.' 'TURBO' 'SANTA ROSA DE OSOS' ... 'CACAHUAL' 'YAVARATÉ'
 'LA GUADALUPE']

Categorías en la columna 'nombre_establecimiento':
['LIC MALLERLAND' 'GIMN NUEVA VILLA MAYOR' 'I.E. SAN VICENTE DEL CONGO'
 ... 'COL RAFAEL POMBO MIS ENCUENTROS'
 'COLEGIO DE BACHILLERATO COLEGIATURA COLOMBIANA'
 'CENT DE EDUCACION PROACTIVA CEP']

Categorías en la columna 'sector':
['NO_OFICIAL' 'OFICIAL']

Cat

- Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños

In [19]:
import unicodedata

# Función para eliminar acentos y caracteres especiales
def remove_accents_and_special_chars(input_str):
    if isinstance(input_str, str):  # Verifica si el valor es una cadena
        # Normalizar la cadena a NFKD
        nfkd_form = unicodedata.normalize('NFKD', input_str)

        # Eliminar acentos
        no_accents = ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

        # Definir caracteres no deseados
        unwanted_chars = [',', ';', '!', '?', '#', '$', '%', '"', "'", '/', '\\', '|', '-']

        # Eliminar caracteres no deseados
        cleaned_str = ''.join([c for c in no_accents if c not in unwanted_chars])

        # Remover espacios en blanco al principio y al final, y convertir a mayúsculas
        result = cleaned_str.strip().upper()

        return result
    else:
        # Si no es una cadena (e.g., NaN o None), devolver el valor tal cual
        return input_str

In [20]:
# Aplicar la función a todas las columnas categóricas
for col in categorical_col:
    preschool_primary_secondary_education_Mineducacion.loc[:, col] = preschool_primary_secondary_education_Mineducacion[col].apply(remove_accents_and_special_chars)


- Codificación de algunas variables categóricas

### Hacer coincidir los códigos de los municipios con el formato de los códigos guardados en la base de datos

Los códigos reales de los municipios de Colombia, están almacenados en la base de datos PostgreSQL del proyecto, en la tabla municipalities dentro del campo dept_mpio_code, junto con la informacion necesaria para georeferenciar todos los municipios y departamentos de Colombia. Este campo guarda el código del municipio en un formato string de exactamente 5 caracteres, los dos primeros corresponden al departamento y los tres restantes al municipio.


Para hacer esta verificación, previamente exportamos desde la base de datos PostgreSQL un DataFrame con los siguientes campos: dept_name, mpio_name y dept_mpio_code, los cuales contienen la información de los departamentos y municipios oficiales, junto con sus respectivos códigos. Este DataFrame se carga en la siguiente celda y se utiliza para comparar con la columna "codmunicipio"

In [21]:
dept_mpios_codes = pd.read_csv("/content/drive/MyDrive/analytics_data_proyect/deptos_mupios.csv", index_col=0, dtype={'dept_mpio_code': str})
print(dept_mpios_codes.info())
dept_mpios_codes.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1121 entries, 0 to 1120
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   dept_mpio_code  1121 non-null   object
 1   dept_name       1121 non-null   object
 2   mupio_name      1121 non-null   object
dtypes: object(3)
memory usage: 35.0+ KB
None


,dept_mpio_code,dept_name,mupio_name
0,97001,VAUPES,MITU
1,97161,VAUPES,CARURU
2,97511,VAUPES,PACOA
3,97666,VAUPES,TARAIRA
4,97777,VAUPES,PAPUNAHUA


 -  Verificar la consistencia de la columna "cod_dane_municipio" en el df preschool_primary_secondary_education_Mineducacion

In [22]:
# Asegurarnos de que todos los valores en 'codigo_m' sean strings
preschool_primary_secondary_education_Mineducacion.loc[:,'cod_dane_municipio'] = preschool_primary_secondary_education_Mineducacion['cod_dane_municipio'].astype(str)

# Calcular la longitud de cada valor en la columna
longitudes = preschool_primary_secondary_education_Mineducacion['cod_dane_municipio'].apply(len)

# Verificar si todas las longitudes son iguales
longitudes.nunique() == 1

False

In [23]:
# Mostrar longitudes únicas (opcional)
print(f"Longitudes únicas: {longitudes.unique()}")

Longitudes únicas: [5 4]


In [24]:
# Contar registros por longitud
long_df = longitudes.value_counts().reset_index()
long_df.rename(columns={'codigo_dane': 'no_dígitos_codigo_dane'}, inplace=True)
long_df['percentage'] = (long_df['count'] / len(longitudes))
long_df.head()

,cod_dane_municipio,count,percentage
0,5,5256461,0.847933
1,4,942686,0.152067


In [25]:
# Mostrar una muestra de registros para cada longitud
for longitud in longitudes.value_counts().index:
    print(f"Muestra de registros con longitud {longitud}:")
    muestra = preschool_primary_secondary_education_Mineducacion[longitudes == longitud].head(5)  # Muestra de los primeros 5 registros
    print(muestra[['cod_dane_municipio']])
    print()

Muestra de registros con longitud 5:
  cod_dane_municipio
0              11001
1              11001
2              11001
4              11001
6              17541

Muestra de registros con longitud 4:
   cod_dane_municipio
3                5837
5                5686
7                5679
8                5686
10               5642



Nota: De lo anterior se notan claras inconsistencias en la columna 'codigo_dane' del df, el 84.8% tiene 5 dígitos (que corresponde con el codigo real que es de 5 dígitos), el 15.2% tiene solamente 4 dígitos (al parecer el cero a la izquierda de los códigos se suprimió).

- Verificar si los codigo_dane de 4 dígitos corresponden a departamentos que se identifican con el 0 + 1 dígito, para validar la teoria de que al generar el dataset se les suprimió el cero a la izquierda

In [26]:
# Filtrar las filas donde 'codigo_dane' tiene 4 dígitos
filtered = preschool_primary_secondary_education_Mineducacion[preschool_primary_secondary_education_Mineducacion['cod_dane_municipio'].str.len() == 4]

# Obtener el primer dígito de cada código y convertirlo a una serie
first_digit = filtered['cod_dane_municipio'].str[0]
# Obtener las categorías únicas de la columna 'departamento'
first_digit.unique()

array(['5', '8'], dtype=object)

Efectivamente ANTIOQUIA Y ATLANTICO son los unicos departamentos que tienen codigo Dane que inicia con un cero + un dígito, 05 y 08 respectivamente.

  - Adicionar un cero a los codigo_dane de 4 dígitos

In [27]:
# Función que agrega un '0' a la izquierda si la longitud del string es 4
def add_zero_if_length_4(codigo):
    if len(codigo) == 4:
        return '0' + codigo
    return codigo

# Aplicar la función a la columna 'codigo'
preschool_primary_secondary_education_Mineducacion.loc[:,'cod_dane_municipio'] = preschool_primary_secondary_education_Mineducacion['cod_dane_municipio'].apply(add_zero_if_length_4)

In [28]:
# Verificar cuantos codigo_dane de 4 dígitos quedaron
len(preschool_primary_secondary_education_Mineducacion[preschool_primary_secondary_education_Mineducacion['cod_dane_municipio'].str.len() == 4])

0

- Comparar que los nombres de los departamentos en cada dataframe esten escritos correctamente

Se aplica la función para racionalizar columnas categoricas (Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños) al df 'dept_mpiios_codes'


In [29]:
# Aplicar función a columnas 'dept_name', 'mupio_name'
for col in dept_mpios_codes[['dept_name', 'mupio_name']]:
    dept_mpios_codes[col] = dept_mpios_codes[col].apply(remove_accents_and_special_chars)

In [30]:
# Función para comparar listas y mostrar diferencias
def compare_lists(df1_col, df2_col, label1, label2):
    # Extraer listas únicas y normalizar
    list1 = set(df1_col.str.strip().str.upper().unique())
    list2 = set(df2_col.str.strip().str.upper().unique())

    # Encontrar diferencias
    only_in_list1 = list1 - list2
    only_in_list2 = list2 - list1

    # Imprimir resultados
    print(f"{label1} que no están en {label2}:")
    print(only_in_list1)

  - Comparar nombres de departamentos en los dataframes

In [31]:
# Comparar listas de departamento
compare_lists(preschool_primary_secondary_education_Mineducacion['departamento'], dept_mpios_codes['dept_name'],
              "Departamentos en preschool_primary_secondary_education_Mineducacion", "Departamentos dept_mpios_codes")

Departamentos en preschool_primary_secondary_education_Mineducacion que no están en Departamentos dept_mpios_codes:
{'CAPITAL BOGOTA D.C.', 'ARCHIPIELAGO DE SAN ANDRESPROVIDENCIA Y SANTA CATALINA', 'GUAJIRA'}


- Ajustamos los nombres de departamento

In [32]:
# Crear el diccionario manualmente
mapeo_departamento = {
    'GUAJIRA': 'LA GUAJIRA',
    'CAPITAL BOGOTA D.C.': 'BOGOTA D.C.',
    'ARCHIPIELAGO DE SAN ANDRESPROVIDENCIA Y SANTA CATALINA': 'ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA'
}

# Hacer mapeo para corregir nombres
preschool_primary_secondary_education_Mineducacion['departamento'] = (
    preschool_primary_secondary_education_Mineducacion['departamento']
    .map(mapeo_departamento)
    .combine_first(preschool_primary_secondary_education_Mineducacion['departamento'])  # Mantiene los nombres originales si no hay mapeo
)

- Comprobar si cambio se produjo

In [33]:
# Comparar listas de departamento
compare_lists(preschool_primary_secondary_education_Mineducacion['departamento'], dept_mpios_codes['dept_name'],
              "Departamentos en preschool_primary_secondary_education_Mineducacion", "Departamentos dept_mpios_codes")

Departamentos en preschool_primary_secondary_education_Mineducacion que no están en Departamentos dept_mpios_codes:
set()


- Verificar que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales

In [34]:
# Comparar listas de códigos
compare_lists(preschool_primary_secondary_education_Mineducacion['cod_dane_municipio'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en preschool_primary_secondary_education_Mineducacion", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en preschool_primary_secondary_education_Mineducacion que no están en Códigos de municipios dept_mpios_codes:
{'94663'}


In [35]:
# A nonbmre de que municipio aparace el codigo que no tiene coincidencia
preschool_primary_secondary_education_Mineducacion[preschool_primary_secondary_education_Mineducacion['cod_dane_municipio'] == '94663']

,anno_inf,departamento,cod_dane_municipio,municipio,codigo_dane,nombre_establecimiento,sector,calendario,nombre_sede,zona,tipo_jornada,grado,especialidad,edad,genero,grupo_etnico,total_matricula
554959,2012,GUAINIA,94663,MAPIRIPANA,294343000034,C.E. DIVINO NINO SALVADOR,OFICIAL,A,C.E. DIVINO NINO SALVADOR,RURAL,MANANA,SEXTO,NO APLICA,13,FEMENINO,NO APLICA,3
555336,2012,GUAINIA,94663,MAPIRIPANA,294001000881,C.E. CARPINTERO,OFICIAL,A,C.E. CARPINTERO SEDE PRINCIPAL,RURAL,MANANA,SEXTO,NO APLICA,13,FEMENINO,NO APLICA,1
555342,2012,GUAINIA,94663,MAPIRIPANA,294001000881,C.E. CARPINTERO,OFICIAL,A,C.E. CARPINTERO SEDE PRINCIPAL,RURAL,MANANA,SEXTO,NO APLICA,13,MASCULINO,NO APLICA,1
555359,2012,GUAINIA,94663,MAPIRIPANA,294001000881,C.E. CARPINTERO,OFICIAL,A,C.E. CARPINTERO SEDE PRINCIPAL,RURAL,MANANA,SEXTO,NO APLICA,14,FEMENINO,NO APLICA,1
555426,2012,GUAINIA,94663,MAPIRIPANA,294343000034,C.E. DIVINO NINO SALVADOR,OFICIAL,A,C.E. DIVINO NINO SALVADOR,RURAL,MANANA,SEXTO,NO APLICA,11,FEMENINO,NO APLICA,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2181418,2012,GUAINIA,94663,MAPIRIPANA,294001000881,C.E. CARPINTERO,OFICIAL,A,C.E. CARPINTERO SEDE PRINCIPAL,RURAL,MANANA,OCTAVO,NO APLICA,19,MASCULINO,INDIGENAS,1
2181435,2012,GUAINIA,94663,MAPIRIPANA,294001000881,C.E. CARPINTERO,OFICIAL,A,C.E. CARPINTERO SEDE PRINCIPAL,RURAL,MANANA,OCTAVO,NO APLICA,17,MASCULINO,INDIGENAS,1
2181552,2012,GUAINIA,94663,MAPIRIPANA,294343000034,C.E. DIVINO NINO SALVADOR,OFICIAL,A,C.E. DIVINO NINO SALVADOR,RURAL,MANANA,OCTAVO,NO APLICA,19,FEMENINO,INDIGENAS,1
2237263,2012,GUAINIA,94663,MAPIRIPANA,294001000881,C.E. CARPINTERO,OFICIAL,A,C.E. CARPINTERO SEDE PRINCIPAL,RURAL,MANANA,TRANSICION,NO APLICA,4,MASCULINO,NEGRITUDES,1


Nota: En la bd del proyecto Mapiripana en Guania, figura como vereda de Barrancaminas hacemos el ajuste

In [36]:
# Reemplazamos cod de MAPIRIPANA por BARRANCOMINAS
preschool_primary_secondary_education_Mineducacion.loc[:, 'cod_dane_municipio'] = preschool_primary_secondary_education_Mineducacion['cod_dane_municipio'].replace({'94663': '94343'})

- Verificar nuevamente que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales

In [37]:
# Comparar listas de códigos
compare_lists(preschool_primary_secondary_education_Mineducacion['cod_dane_municipio'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en preschool_primary_secondary_education_Mineducacion", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en preschool_primary_secondary_education_Mineducacion que no están en Códigos de municipios dept_mpios_codes:
set()


### Procesamiento final como preparación para integrarlo a la bd de datos del proyecto

In [38]:
# Eliminar columnas 'departamento', 'municipio' que ya no son necesarias
columns_to_drop = ['departamento', 'municipio']
final_preschool_primary_secondary_education_Mineducacion = preschool_primary_secondary_education_Mineducacion.drop(columns=columns_to_drop)

In [39]:
# Adicionar columna para trazabilidad de la fuente
final_preschool_primary_secondary_education_Mineducacion['source_id'] = 115

In [40]:
final_preschool_primary_secondary_education_Mineducacion.columns

Index(['anno_inf', 'cod_dane_municipio', 'codigo_dane',
       'nombre_establecimiento', 'sector', 'calendario', 'nombre_sede', 'zona',
       'tipo_jornada', 'grado', 'especialidad', 'edad', 'genero',
       'grupo_etnico', 'total_matricula', 'source_id'],
      dtype='object')

In [41]:
# Ajustar nombre de columnas

# Definir el diccionario de traducción
translation_map = {
    'anno_inf':'year',
    'cod_dane_municipio': 'dane_code',
    'codigo_dane': 'educational_institution_dane_code',
    'nombre_establecimiento': 'educational_institution_name',
    'sector': 'sector',
    'calendario': 'calendar',
    'nombre_sede': 'institution_name',
    'zona': 'area',
    'tipo_jornada': 'shift_type',
    'grado': 'level',
    'especialidad': 'speciallity',
    'edad': 'age',
    'genero': 'gender',
    'grupo_etnico': 'ethnic_group',
    'total_matricula': 'total_enrollment',
    'source_id': 'source_id'
}

# Renombrar las columnas
final_preschool_primary_secondary_education_Mineducacion.rename(columns=translation_map, inplace=True)

In [42]:
#Estructura final del dataset a integrar a la base de datos
final_preschool_primary_secondary_education_Mineducacion.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6199147 entries, 0 to 6199999
Data columns (total 16 columns):
 #   Column                             Dtype 
---  ------                             ----- 
 0   year                               int64 
 1   dane_code                          object
 2   educational_institution_dane_code  int64 
 3   educational_institution_name       object
 4   sector                             object
 5   calendar                           object
 6   institution_name                   object
 7   area                               object
 8   shift_type                         object
 9   level                              object
 10  speciallity                        object
 11  age                                int64 
 12  gender                             object
 13  ethnic_group                       object
 14  total_enrollment                   int64 
 15  source_id                          int64 
dtypes: int64(5), object(11)
memory usage: 804

## Salvar en archivo csv en el drive

In [43]:
# Guardar en archivos CSV en el drive
final_preschool_primary_secondary_education_Mineducacion.to_csv('/content/drive/MyDrive/analytics_data_proyect/initial_transformation/preschool_primary_secondary_education_Mineducacion.csv', index=False)